<a href="https://colab.research.google.com/github/Brunonicolas-creator/WebScraping/blob/main/WebScrap_MercLib_Deptos_Posadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalo librerias
!pip install requests
!pip install BeautifulSoup

In [66]:
#Las llamo
import requests
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import numpy as np

In [67]:
#Pag Semilla
r= requests.get('https://inmuebles.mercadolibre.com.ar/departamentos/venta/misiones/posadas/')

In [ ]:
r.status_code

In [69]:
#Parseo con BS
soup= BeautifulSoup(r.content,'html.parser')

**PARA SCRAPEAR SOLO LA PRIMERA PAGINA**

In [71]:
#Busco los titulos e inspecciono la clase
titulos=soup.find_all('h2',attrs={"class":"ui-search-item__title"})
#Solo quiero el texto
titulos= [i.text for i in titulos]

In [72]:
#Armo el arbol para llegar al simbolo del precio
dom=etree.HTML(str(soup))
simb_precios=dom.xpath('//li[@class="ui-search-layout__item"]//div[@class="ui-search-result__wrapper"]//div[@class="ui-search-item__group__element ui-search-price__part-without-link"]//div[@class="ui-search-price ui-search-price--size-medium"]//div[@class="ui-search-price__second-line"]//span[@class="price-tag ui-search-price__part"]//span[@class="price-tag-symbol"]')
#Solo quiero el texto
simb_precios=[i.text for i in simb_precios]

In [73]:
#Armo el arbol para llegar al precio
dom=etree.HTML(str(soup))
precios=dom.xpath('//li[@class="ui-search-layout__item"]//div[@class="ui-search-result__wrapper"]//div[@class="ui-search-item__group__element ui-search-price__part-without-link"]//div[@class="ui-search-price ui-search-price--size-medium"]//div[@class="ui-search-price__second-line"]//span[@class="price-tag ui-search-price__part"]//span[@class="price-tag-fraction"]')
#Solo quiero el texto
precios=[i.text for i in precios]

In [74]:
#Armo el arbol para llegar a caract
dom=etree.HTML(str(soup))
carac=dom.xpath('//div[@class="ui-search-result__content-wrapper"]//div[@class="ui-search-item__group ui-search-item__group--attributes"]//ul[@class="ui-search-card-attributes ui-search-item__group__element"]//li[@class="ui-search-card-attributes__attribute"]')
#Solo quiero el texto
carac=[i.text for i in carac[0:96:2]]


In [75]:
#Convierto en un DataFrame
df=pd.DataFrame({"titulo":titulos,"simbolo":simb_precios,"precios":precios,"carac":carac})

In [76]:
#Elimino caracteres en columna carac, para que solo me queden mts2
df["carac"] = df["carac"].replace({'cubiertos':''}, regex=True)
df["carac"]=df["carac"].replace({'totales':''}, regex=True)
df["carac"]=df["carac"].replace({'m²':''}, regex=True)

In [ ]:
#Exportar a csv
df.to_csv('deptosposadas.csv')
from google.colab import files 
files.download('deptosposadas.csv')

**PARA SCRAPEAR DE FORMA HORIZONTAL**

In [ ]:
#Boton de siguiente
siguiente=dom.xpath('//div[@class="ui-search-pagination"]/ul/li[contains(@class,"--next")]/a')[0].get('href')
siguiente

In [ ]:
#Boton inicial (pagina 1)
ini= soup.find('span',attrs={"class":"andes-pagination__link"}).text
#Lo paso a un entero
ini= int(ini)
ini


In [ ]:
#Traigo las otras paginas
can=soup.find('li',attrs={"class":"andes-pagination__page-count"})
can=int(can.text.split(" ")[1])
can

In [80]:
#Armo una lista de cada seccion
lista_titulos=[]
lista_simb_precios=[]
lista_precios=[]
lista_carac=[]


In [ ]:
#Funcion completa
siguiente='https://inmuebles.mercadolibre.com.ar/departamentos/venta/misiones/posadas/'
while True:
    r = requests.get(siguiente)
    if r.status_code==200:
        soup= BeautifulSoup(r.content,'html.parser')
        #Titulos
        titulos=soup.find_all('h2',attrs={"class":"ui-search-item__title"})
        titulos= [i.text for i in titulos]
        lista_titulos.extend(titulos)
        #Simbolo
        dom=etree.HTML(str(soup))
        simb_precios=dom.xpath('//li[@class="ui-search-layout__item"]//div[@class="ui-search-result__wrapper"]//div[@class="ui-search-item__group__element ui-search-price__part-without-link"]//div[@class="ui-search-price ui-search-price--size-medium"]//div[@class="ui-search-price__second-line"]//span[@class="price-tag ui-search-price__part"]//span[@class="price-tag-symbol"]')
        simb_precios=[i.text for i in simb_precios]
        lista_simb_precios.extend(simb_precios)
        #Precios
        dom=etree.HTML(str(soup))
        precios=dom.xpath('//li[@class="ui-search-layout__item"]//div[@class="ui-search-result__wrapper"]//div[@class="ui-search-item__group__element ui-search-price__part-without-link"]//div[@class="ui-search-price ui-search-price--size-medium"]//div[@class="ui-search-price__second-line"]//span[@class="price-tag ui-search-price__part"]//span[@class="price-tag-fraction"]')
        precios=[i.text for i in precios]
        lista_precios.extend(precios)
        #Carac
        dom=etree.HTML(str(soup))
        carac=dom.xpath('//div[@class="ui-search-result__content-wrapper"]//div[@class="ui-search-item__group ui-search-item__group--attributes"]//ul[@class="ui-search-card-attributes ui-search-item__group__element"]//li[@class="ui-search-card-attributes__attribute"]')  
        carac=[i.text for i in carac[0:96:2]]
        lista_carac.extend(carac)
        ini= soup.find('span',attrs={"class":"andes-pagination__link"}).text
        ini= int(ini)
        can=soup.find('li',attrs={"class":"andes-pagination__page-count"})
        can=int(can.text.split(" ")[1])
    else:
        break
    print(ini,can)
    if ini==can:
        break
    siguiente=dom.xpath('//div[@class="ui-search-pagination"]/ul/li[contains(@class,"--next")]/a')[0].get('href')

In [82]:
#Veo que tenga la misma cantidad de len para llevarlo a un DF
print(len(lista_precios))
print(len(lista_simb_precios))
print(len(lista_carac))
print(len(lista_titulos))

195
195
195
195


In [84]:
#Transformo en un DataFrame
df=pd.DataFrame({"titulo":lista_titulos,"simbolo":lista_simb_precios,"precios":lista_precios,"carac":lista_carac})

In [85]:
#Elimino caracteres en columna carac, para que solo me queden mts2
df["carac"] = df["carac"].replace({'cubiertos':''}, regex=True)
df["carac"]=df["carac"].replace({'totales':''}, regex=True)
df["carac"]=df["carac"].replace({'m²':''}, regex=True)

In [86]:
#Exportar a csv
df.to_csv('deptosposadas.csv')
from google.colab import files 
files.download('deptosposadas.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>